<a href="https://colab.research.google.com/github/nwang2208/Bioinformatics-Class-Projects/blob/mapping-reads/Final4A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
k = 10
m = 10

In [ ]:
!pip install Bio

In [ ]:
from Bio import SeqIO
import pickle
import numpy as np

Opening stuff

In [ ]:
genome_dict = {}
for i in range(999):
    for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/project4a-data/project4a_10000_genome_'+str(i)+'.fasta', "fasta"):
        genome_dict[seq_record.id] = str(seq_record.seq)

In [ ]:
genome_kmer_dict = {}
for key, value in genome_dict.items():
    transcripts = []
    for i in range(0, len(value)-k+1):
        seq = value[i:i+k]
        transcripts.append(seq)
    genome_kmer_dict[key] = transcripts

In [ ]:
reads = {}
for seq_record in SeqIO.parse('/content/drive/MyDrive/122data/project4a-data/project4a_10000_reads.fasta', "fasta"):
    reads[seq_record.id] = str(seq_record.seq)

Minimizing

In [ ]:
def minKmer(read):
    kmers_list = []
    for i in range(0,len(read)-m+1):
        seq = read[i:i+m]
        kmers_list.append(seq)
    final = min(x for x in kmers_list if isinstance(x, str))
    return final,kmers_list

In [ ]:
min_dict = {}
kmer_dict = {}
for key, value in reads.items():
    final = minKmer(value)
    min_dict[key] = final[0]
    kmer_dict[key] = final[1]

Open map_dict

In [ ]:
map_dict = {}
for key1, value1 in min_dict.items():
    for key2, value2 in genome_dict.items():
        if key1 in map_dict.keys() and value1 in value2:
            map_dict[key1].append(key2)
        elif value1 in value2:
            map_dict[key1] = [key2]

from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
import pickle

with open('data.pickle', 'wb') as handle:
    pickle.dump(map_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'data.pickle'}
media = MediaFileUpload('data.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()

In [ ]:
map_dict = open('/content/drive/MyDrive/data.pickle', 'rb')
map_dict = pickle.load(map_dict)

In [ ]:
map_dict

In [ ]:
map_dict2 = {}
for key1, value1 in min_dict.items():
    for key2, value2 in genome_dict.items():
        if key1 in map_dict2.keys() and value1 in value2:
            map_dict2[key1].append(key2)
        elif value1 in value2:
            map_dict2[key1] = [key2]

Map to genomes

In [ ]:
def transcriptTime(genome):
    start = 0
    end = 0
    transcript={}
    for i in range(0, len(genome)):
        end = i + k
        if(end > len(genome)):
          break
        seq = genome[i:end]
        if seq in transcript.keys():
          transcript[seq].append(i)
        else:
          transcript[seq] = [i]

    return(transcript)

In [ ]:
def scoring(positions): #helps alignRead

    #first find the ranges
    r1 = {}
    r2 = {}
    r3 = {}



    for x in range(0, len(positions)):
        for y in range(0, len(positions[x])):
            pos = positions[x][y]
            if len(str(pos)) == 3: #3 digit number
                rg = int(str(pos)[:2] + str(0))
            elif len(str(pos)) == 2: #2 digit number
                rg = int(str(pos)[0] + str(0))
            else: #single digit
                rg = pos

            if(x < 15):
                if rg in r1.keys():
                    r1[rg] += 1
                else:
                    r1[rg] = 1
            elif(x < 30):
                if rg in r2.keys():
                    r2[rg] += 1
                else:
                    r2[rg] = 1
            else:
                if rg in r3.keys():
                    r3[rg] += 1
                else:
                    r3[rg] = 1



    #now score
    final_positions = []
    scores = {}
    ranges = {}
    one_score = 0
    for x in range(0, len(positions)):
          if len(positions[x]) > 1: #there are multiple positions to choose from
              for i in range(0, len(positions[x])):
                  if(len(str(positions[x][i])) == 3): #3 digit number
                      kee = int(str(positions[x][i])[:2] + str(0))
                  elif len(str(positions[x][i])) == 2: #2 digit number
                      kee = int(str(positions[x][i])[0] + str(0))
                  else: #single digit number
                      kee = positions[x][i]

                  kee_before = kee - 10
                  kee_after = kee + 10

                  if(x < 15):
                      ranges = r1.copy()
                  elif(x < 30):
                      ranges = r2.copy()
                  else:
                      ranges = r3.copy()




                  if(kee_before in ranges.keys()):
                      score1 = ranges[kee_before]
                      one_score += score1
                  if(kee in ranges.keys()):
                      score2 = ranges[kee]
                      one_score += score2
                  if(kee_after in ranges.keys()):
                      score3 = ranges[kee_after]
                      one_score += score3

                  scores[positions[x][i]] = one_score
                  one_score = 0
              best_pos = maxDict(scores)
              final_positions.append(best_pos)
              scores = {}
          else:
              final_positions.append(positions[x][0])

    return(final_positions)

In [ ]:
def screening(final_positions, positions): #if there are consecutive positions and they accidentally chose the wrong ones, let's bring back incrememnting consecutive positions
    for r in range(1, len(final_positions)-1):
        num_interest = final_positions[r]
        before_num = final_positions[r - 1]
        after_num = final_positions[r + 1]
        if((num_interest != before_num+1) and (num_interest != after_num-1)): #both sides aren't good
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1 and positions[r][i] == after_num-1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != before_num+1 and after_num == 10000): #after is a non match
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != after_num-1 and before_num == 10000): #before is nonmatch
            for i in range(0, len(positions[r])):
                if(positions[r][i] == after_num-1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000
        elif(num_interest != before_num+1 and after_num != 10000 and before_num != 10000): #let's try to match our nums
            for i in range(0, len(positions[r])):
                if(positions[r][i] == before_num+1): #ranges didn't catch things
                    final_positions[r] = positions[r][i]
                    break
                else:
                    final_positions[r] = 10000

    #now go through the list again to make sure no false 10000
    for a in range(1, len(final_positions) - 1):
        num_interest = final_positions[a]
        before_num = final_positions[a - 1]
        after_num = final_positions[a + 1]

        if (num_interest == 10000 and before_num != 10000): #before is a normal number
            for j in range(0, len(positions[a])):
                if(positions[a][j] == before_num + 1):
                    final_positions[a] = positions[a][j]
                    break
        elif(num_interest == 10000 and after_num != 10000): #after is a normal number
            for j in range(0, len(positions[a])):
                if(positions[a][j] == after_num - 1):
                    final_positions[a] = positions[a][j]
                    break




#we need to look at the last base too
    #if(final_positions[len(final_positions) - 1] != final_positions[len(final_positions)-2]):
        #for i in range(0, len(positions[r])):
                #if(positions[len(final_positions) - 1][i] == final_positions[len(final_positions)-2]): #ranges didn't catch things
                    #final_positions[len(final_positions) - 1] = positions[len(final_positions) - 1][i]
                    #break
                #else:
                    #final_positions[len(final_positions) - 1] = 10000


    return(final_positions)

In [ ]:
def maxDict(d): #helps(alignRead)
  return max(d, key = d.get)

In [ ]:
def alignRead(kmer_list, genome): #final positions assigns a position to all bases, whether or not that is a good thing
    start = 0
    end = 0
    positions = []
    transcript = transcriptTime(genome)

    for seq in kmer_list:
        if seq in transcript.keys(): #match with ref
            positions.append(transcript[seq])
        else:
            positions.append([10000])

    positions2 = scoring(positions)
    final_positions = screening(positions2, positions)


    return(final_positions)

In [ ]:
def splitList(final_positions):
    #screen out just wrong numbers
    split_list = {}
    count = 0
    for pos in final_positions:
        if(pos != 10000):
            if(count in split_list.keys()):
                split_list[count].append(pos)
            else:
                split_list[count] = [pos]
        elif(pos == 10000):
              count += 1
              if(count-1 in split_list.keys()):
                  a = 'nothing'
              else:
                  count -= 1

    return(split_list)

In [ ]:
def matchPos(split_list): #maybe just skip ahead of final_positions when you hit 1000 and see if you hit a good position a certain amount of kmers away
    match_pos = {}
    match_pos[0] = split_list[0].copy()
    match_key = 0

    #creating match_pos, a dict with only positions of bases that are right, this only works for substituion rn
    keeplast = 0
    prev_list = 0
    prev_index = 0
    for t in range(1, len(split_list)):
        firstofnext = split_list[t][0]
        if (keeplast != 1):
            prev_list = t-1
            prev_index = len(split_list[prev_list]) - 1
            keeplast = 0

        lastofprev = split_list[prev_list][prev_index]

        #both del and in look the same in matchPos
        if(0 <= firstofnext - lastofprev <= k*3):
            match_key += 1
            match_pos[match_key] = split_list[t]
            keeplast = 0
        else:
            keeplast = 1

    return(match_pos)

In [ ]:
def bestGenome(matchposlist):
    score_genomes = {}
    for key, value in matchposlist.items():
        score = 0
        for key2, value2 in value.items():
            score += len(value2)
        score_genomes[key] = score

    genomes_list = []
    big = max(score_genomes.values())
    for key, value in score_genomes.items():
          if(value == big):
              genomes_list.append(key)

    return(genomes_list)





In [ ]:
def everythingMinWork(read_name, genome_list):

    test_matches = {}



    for el in genome_list:
        final_positions = alignRead(kmer_dict[read_name], genome_dict[el])
        split_list = splitList(final_positions)
        if(len(split_list) > 0):
            match_pos = matchPos(split_list)
        else:
            match_pos = {}

        test_matches[el] = match_pos #genome name | all the positions

    for key, value in test_matches.items():
        if(len(value) > 0):
            return bestGenome(test_matches)
        else:
            return "mismatch"


In [ ]:
def everythingMinNoWork(read_name):
      test_matches = {}

      for key in genome_dict:
          final_positions = alignRead(kmer_dict[read_name], genome_dict[key])
          split_list = splitList(final_positions)
          if(len(split_list) > 0):
              match_pos = matchPos(split_list)
          else:
              match_pos = {}

          test_matches[key] = match_pos

      return bestGenome(test_matches)



In [ ]:
def everything(read_name, top10):
      test_matches = {}
      for el in top10:
          final_positions = alignRead(kmer_dict[read_name], genome_dict[el])
          split_list = splitList(final_positions)
          if(len(split_list) > 0):
              match_pos = matchPos(split_list)
          else:
              match_pos = {}

          test_matches[el] = match_pos

      return bestGenome(test_matches)


Testing out something...(meh)

In [ ]:
reads4 = {}
for i in range(0, 100):
    reads4['read_'+str(i)] = reads['read_'+str(i)]


In [ ]:
mtog_dict4 = {}


for key in reads4:
    if key in map_dict.keys():
        pos = everythingMinWork(key)
        if(pos != "mismatch"):
            mtog_dict4[key] = pos
        else:
            pos = everythingMinNoWork(key)
            mtog_dict4[key] = pos
    else:
          pos = everythingMinNoWork(key)
          mtog_dict4[key] = pos



In [ ]:
mtog_dict4

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
import pickle

with open('mtog4.pickle', 'wb') as handle:
    pickle.dump(mtog_dict4, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'mtog4.pickle'}
media = MediaFileUpload('mtog4.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()

Pick out top 10 genomes

In [ ]:
map_dict

In [ ]:
count_dict = {}
for key, value in map_dict.items():
    for el in value:
        if el in count_dict.keys():
            count_dict[el] += 1
        else:
            count_dict[el] = 1


In [ ]:
sorted_dict = sorted(count_dict.items(), key=lambda x:x[1])
sorted_dict = dict(sorted_dict)


In [ ]:
sorted_dict

In [ ]:
sorted_keys = list(sorted_dict.keys())

In [ ]:
len(sorted_keys)

999

In [ ]:
top10 = sorted_keys[990:1000]

In [ ]:
top10

['Genome_Number_634',
 'Genome_Number_659',
 'Genome_Number_671',
 'Genome_Number_353',
 'Genome_Number_146',
 'Genome_Number_377',
 'Genome_Number_622',
 'Genome_Number_489',
 'Genome_Number_358']

In [ ]:
top10_dict = {}

for i in range(9):
    top10_dict[top10[i]] = i

In [ ]:
top10_dict

Map all reads to the top 10 genomes

In [ ]:
final_dict = {}

for key in reads:
    final_dict[key] = everything(key, top10)


from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaFileUpload
import pickle

with open('final.pickle', 'wb') as handle:
    pickle.dump(final_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#Authenticate and construct the Drive API client
auth.authenticate_user()
drive_service = build('drive', 'v3')

file_metadata = {'name': 'final.pickle'}
media = MediaFileUpload('final.pickle', mimetype='application/octet-stream')
file = drive_service.files().create(body=file_metadata, media_body=media,
                                    fields='id').execute()


In [ ]:
final_dict = open('/content/drive/MyDrive/final.pickle', 'rb')
final_dict = pickle.load(final_dict)

In [ ]:
final_dict

Make sure each read maps to 1 genome

In [ ]:
ans_dict = {}

for key, value in final_dict.items():
    if(len(value) > 1):
        max_num = top10_dict[value[0]]
        max = value[0]
        for i in range(1, len(value)):
            if(top10_dict[value[i]] > max_num):
                max_num = top10_dict[value[i]]
                max = value[i]
        ans_dict[">"+key] = max
    elif(len(value) == 1):
        ans_dict[">"+key] = value[0]
    else:
        ans_dict[">"+key] = 'Genome_Number_358'

In [ ]:
len(ans_dict)

20000

In [ ]:
len(reads)

20000

In [ ]:
ans_dict

In [ ]:
ans = [0] * 20000

i = 0
for key, value in ans_dict.items():
    ans[i] = key + ' ' + value
    i+=1


In [ ]:
ans

In [ ]:
np.savetxt("predictions.csv",
           ans,
           delimiter =" ",  # Set the delimiter as a comma followed by a space
           fmt ='% s')